In [1]:
import pints
import pints.toy
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras

import emupints
import emupints.plot as emuplt
import emupints.utils as emutils
import emupints.metrics as emumet


from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Prediction time and mae depending on Kernel and problem

In [2]:
def create_NN_emulator(log_likelihood, X, y, model_size):
    # save best val model
    cp_callback = keras.callbacks.ModelCheckpoint("models/nn_performance.hdf5", 
                                                  monitor='val_mean_absolute_error', 
                                                  save_best_only=True, 
                                                  verbose=0,
                                                  save_weights_only=True)
    
    emu = emupints.NNEmulator(log_likelihood, X, y, 
                              model_size=model_size,
                              input_scaler=MinMaxScaler(feature_range = (-1, 1)),
                              output_scaler=StandardScaler(),
                             )
    emu.set_parameters()
    emu.fit(verbose=0, callbacks=[cp_callback])
    # restore best
    emu._model.load_weights("models/nn_performance.hdf5")
    
    return emu

In [3]:
nn_models = ['small', 'average', 'large']

train_size = 2000
test_size = 400

problems = [
    (emupints.Problems.LogisticModel, 'Logistic'),
    (emupints.Problems.SIRModel, 'SIR'),
    (emupints.Problems.FitzhughNagumoModel, 'FitzhughNagumo'),
    (emupints.Problems.LotkaVolterraModel, 'LotkaVolterra'),
    (emupints.Problems.HodgkinHuxleyIKModel, 'HodgkinHuxleyIK'),
    (emupints.Problems.GoodwinOscillatorModel, 'GoodwinOscillator'),
]

results = {}

for problem, problem_name in problems:
    print(problem_name)
    # load problem
    problem = emupints.Problems.load_problem(problem)
    n_parameters = problem['n_parameters']
    log_likelihood = problem['log_likelihood']
    log_posterior = problem['log_posterior']
    bounds = problem['bounds']
    parameters = problem['parameters']

    # create data for training and testing
    # generate examples using normal distribution around real parameters
    train_X = parameters + np.random.randn(train_size, n_parameters) * (0.05 * parameters)
    train_y = np.apply_along_axis(log_likelihood, 1, train_X)

    test_X = parameters + np.random.randn(test_size, n_parameters) * (0.05 * parameters)
    test_y = np.apply_along_axis(log_likelihood, 1, test_X)

    # metrics to store
    model_chain_mae = []
    model_chain_mape = []
    model_time = []

    for model_size in nn_models:
        # train emulator
        emu = create_NN_emulator(log_likelihood, train_X, train_y, model_size)

        # Perform single MCMC run to calcluate chain mape
        mcmc = pints.MCMCSampling(
            log_posterior, 
            1, # one chain
            [parameters], 
        )
        mcmc.set_max_iterations(20000)
        mcmc.set_log_to_screen(False)
        mcmc_chain = mcmc.run()[0]
    
        # accuracy via mae
        chain_mae = emumet.chain_mae(mcmc_chain, emu, log_likelihood)
        model_chain_mae.append(chain_mae)
        
        # accuracy via absolute error along a chain 
        chain_mape = emumet.chain_mape(mcmc_chain, emu, log_likelihood)
        model_chain_mape.append(chain_mape)

        # speed
        pred_time = %timeit -r 50 -o -q emu(parameters)
        pred_time = pred_time.average
        model_time.append(pred_time)
        
 
        # delete used emulators
        del emu
        
        print("    {}: {:.5f}, {:.5f}, {:.10f}".format(model_size, chain_mae, chain_mape, pred_time))
    
    # store results
    results[problem_name] = [model_chain_mae, model_chain_mape, model_time]

Logistic
    small: 0.54457, 0.00059, 0.0004780007
    average: 2.96379, 0.00318, 0.0005201673
    large: 3.15528, 0.00339, 0.0004798853
SIR
    small: 7.52415, 0.01216, 0.0004530781
    average: 29.09075, 0.04549, 0.0005011633
    large: 27.58551, 0.04324, 0.0005433427
FitzhughNagumo
    small: 3.81762, 0.01645, 0.0004537895
    average: 3.19439, 0.01328, 0.0005135470
    large: 7.48213, 0.03275, 0.0005091286
LotkaVolterra
    small: 32.85583, 0.06612, 0.0004787625
    average: 85.58501, 0.19266, 0.0005573160
    large: 76.27665, 0.16818, 0.0005432155
HodgkinHuxleyIK
    small: 20.05818, 0.01399, 0.0005381487
    average: 38.09448, 0.02627, 0.0005820103
    large: 23.19937, 0.01642, 0.0005973458
GoodwinOscillator
    small: 43.10572, 0.01766, 0.0007213356
    average: 95.57805, 0.04000, 0.0006191102
    large: 145.14267, 0.06200, 0.0006028620


In [4]:
# print results 
max_name_len = max([len(name) for _, name in problems])

columns = "".join([name.center(10) for name in nn_models])
columns = " " * (max_name_len + 8) + columns
print(columns)
print("-" * 70)

for _, problem_name in problems:
    print(problem_name.center(max_name_len), "|mae  |", end="")
    chain_maes = np.array(results[problem_name][0]) # don't convert
    for mae in chain_maes:
        print("{:.2f}".format(mae).center(10), end="")
    print()
    
    print("".rjust(max_name_len),"|cmape|", end="")
    chain_cmaes = np.array(results[problem_name][1]) * 100 # convert to percentages
    for cmae in chain_cmaes:
        print("{:.2f}".format(cmae).center(10), end="")
    print()
    
    print("".rjust(max_name_len), "|time |", end="")
    times = np.array(results[problem_name][2]) * 1000000 # convert to micro-seconds
    for time in times:
        print("{:.2f}".format(time).center(10), end="")
    print()
    
    print("-" * 70)

                           small    average    large   
----------------------------------------------------------------------
     Logistic     |mae  |   0.54      2.96      3.16   
                  |cmape|   0.06      0.32      0.34   
                  |time |  478.00    520.17    479.89  
----------------------------------------------------------------------
       SIR        |mae  |   7.52     29.09     27.59   
                  |cmape|   1.22      4.55      4.32   
                  |time |  453.08    501.16    543.34  
----------------------------------------------------------------------
  FitzhughNagumo  |mae  |   3.82      3.19      7.48   
                  |cmape|   1.64      1.33      3.27   
                  |time |  453.79    513.55    509.13  
----------------------------------------------------------------------
  LotkaVolterra   |mae  |  32.86     85.59     76.28   
                  |cmape|   6.61     19.27     16.82   
                  |time |  478.76    557.32 

In [1]:
# convenient print to copy paste result to Latex
problem_names = ['Logistic', 'SIR', 'FitzhughNagumo', 'LotkaVolterra', 'HodgkinHuxleyIK', 'GoodwinOscillator']
for problem_name in problem_names:
    for i, model_size in enumerate(nn_models):
        print(problem_name, model_size, end=": ")
        chain_mae = results[problem_name][0][i]
        chain_cmae = results[problem_name][1][i] * 100
        time = results[problem_name][2][i] * 1000000
        print("& {:.2f} & {:.2f} & {:.2f}".format(chain_mae, chain_cmae, time))

NameError: name 'nn_models' is not defined